# Examples and Exercises from Think Stats, 2nd Edition

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT


In [2]:
from __future__ import print_function, division
import math

import nsfg

## Examples from Chapter 1

Read NSFG data into a Pandas DataFrame.

In [3]:
preg = nsfg.ReadFemPreg()
preg.head()

FileNotFoundError: [Errno 2] No such file or directory: '2002FemPreg.dct'

Print the column names.

In [ ]:
preg.columns

Select a single column name.

In [ ]:
preg.columns[1]

Select a column and check what type it is.

In [ ]:
pregordr = preg['pregordr']
type(pregordr)

Print a column.

In [ ]:
pregordr

Select a single element from a column.

In [ ]:
pregordr[0]

Select a slice from a column.

In [ ]:
pregordr[2:5]

Select a column using dot notation.

In [ ]:
pregordr = preg.pregordr

Count the number of times each value occurs.

In [ ]:
preg.outcome.value_counts().sort_index()

Check the values of another variable.

In [ ]:
preg.birthwgt_lb.value_counts().sort_index()

Make a dictionary that maps from each respondent's `caseid` to a list of indices into the pregnancy `DataFrame`.  Use it to select the pregnancy outcomes for a single respondent.

In [ ]:
caseid = 10229
preg_map = nsfg.MakePregMap(preg)
indices = preg_map[caseid]
preg.outcome[indices].values

## Exercises

Select the `birthord` column, print the value counts, and compare to results published in the [codebook](http://www.icpsr.umich.edu/nsfg6/Controller?displayPage=labelDetails&fileCode=PREG&section=A&subSec=8016&srtLabel=611933)

In [ ]:
preg['birthord'].value_counts().sort_index()

We can also use `isnull` to count the number of nans.

In [ ]:
preg.birthord.isnull().sum()

Select the `prglngth` column, print the value counts, and compare to results published in the [codebook](http://www.icpsr.umich.edu/nsfg6/Controller?displayPage=labelDetails&fileCode=PREG&section=A&subSec=8016&srtLabel=611931)

In [ ]:
preg['prglngth']

To compute the mean of a column, you can invoke the `mean` method on a Series.  For example, here is the mean birthweight in pounds:

In [ ]:
preg.totalwgt_lb.mean()

Create a new column named <tt>totalwgt_kg</tt> that contains birth weight in kilograms.  Compute its mean.  Remember that when you create a new column, you have to use dictionary syntax, not dot notation.

In [ ]:
preg['totalwgt_kg'] = preg['totalwgt_lb']*0.453592

In [ ]:
preg['totalwgt_kg']

`nsfg.py` also provides `ReadFemResp`, which reads the female respondents file and returns a `DataFrame`:

In [ ]:
resp = nsfg.ReadFemResp()

`DataFrame` provides a method `head` that displays the first five rows:

In [ ]:
resp.head()

Select the `age_r` column from `resp` and print the value counts.  How old are the youngest and oldest respondents?

In [ ]:
## the youngest is 15 and oldest is 44
resp['age_r'].value_counts().sort_index()

We can use the `caseid` to match up rows from `resp` and `preg`.  For example, we can select the row from `resp` for `caseid` 2298 like this:

In [ ]:
resp[resp.caseid==2298]

And we can get the corresponding rows from `preg` like this:

In [ ]:
preg[preg.caseid==2298]

How old is the respondent with `caseid` 1?

In [ ]:
## the person with caseid 1 is 44 years old.
one_persons_data = resp[resp.caseid==1]
one_persons_data["age_r"]

What are the pregnancy lengths for the respondent with `caseid` 2298?


In [ ]:
one_persons_data = preg[preg.caseid==2298]
one_persons_data["prglngth"]

What was the birthweight of the first baby born to the respondent with `caseid` 5012?

In [ ]:
## It turns out the person with caseid 5012 only had the one child. The baby was 6.0 lbs.
one_persons_data = preg[preg.caseid == 5012]
one_persons_data["birthwgt_lb"]

In [ ]:
## testing chapter 2 ideas.
## It seemed to work but we may need to remove Null values. Pressing on!
hist = {}
for x in preg.birthwgt_lb:
    hist[x] = hist.get(x,0)+1

## I used this to check my work but it spams nans so I commented it out
# hist

In [ ]:
## kind of an elegant way of doing this. Seems to get the same results.
from collections import Counter
counter = Counter(preg.birthwgt_lb)

## Since this was practice I commented this out so I would not spam nans :)
#counter

In [ ]:
## looks like thinkstats2 ignored the null values.
import thinkstats2
hist = thinkstats2.Hist(preg.birthwgt_lb)

## Same with this one I commented it out so my code would be more readable.
# hist

In [ ]:
## this looks like it checks the frequencies based on the weight I put in.
## So in this case I tried 6 lbs which was the same weight that caseid = 5012 had
## It turns out that there were 2223 babies that also had this weight.
hist.Freq(6)



In [ ]:
## Lets check something impossible and see what we get.
## Confirmed there were no babies that weighed 100 lbs.
hist.Freq(100)


In [ ]:
## Since this is a dictionary this gives me the values stored in the dictionary.
hist.Values()


In [ ]:
## Very cool this is starting to look like a histogram.
for val in sorted(hist.Values()):
    print(val,hist.Freq(val))



In [ ]:
## Awesome we got a histogram that looks fairly normal.
import thinkplot
thinkplot.Hist(hist)
thinkplot.Show(xlabel = 'Weight in lbs', ylabel='Frequency')



In [ ]:
## following the book I already imported the data and made a column with weights in lb
## this must be used to ensure that we are not building bias into our histogram since babies that die are most likely underweight. :(
live=preg[preg.prgoutcome==1]



In [ ]:
## Interesting it looks very similar to what I got before but I did not filter for live births.
## I changed my variable to hist2 just to be sure
hist_birthwgt_lb = thinkstats2.Hist(live.birthwgt_lb,label= 'birthwgt_lb')
thinkplot.Hist(hist_birthwgt_lb)
thinkplot.Show(xlabel='Weight in lbs', ylabel = 'Frequency')

In [ ]:
## The author brings up a good point outliers would be easy to miss on a frequency graph.
## here is the original with with outliers still in the histogram
hist_prgLngth = thinkstats2.Hist(live.prglngth,label="Weeks until birth")
thinkplot.Hist(hist_prgLngth)
thinkplot.Show(xlabel = 'Weeks', ylabel = 'frequency')

In [ ]:
## Checking for outliers
for weeks, freq in hist_prgLngth.Smallest(10):
    print(weeks, freq)


In [ ]:
## now we are making a list of the babies who were either first borns or not
firsts = live[live.birthord == 1]
others = live[live.birthord != 1]

## building two histograms one will be a histogram for first borns the other will contain all others.
first_hist = thinkstats2.Hist(firsts.prglngth)
other_hist = thinkstats2.Hist(others.prglngth)

## plotting on the same axis so we can compare them.
## I made an observation here that the histograms are hard to read due to a difference in sample size.
## The book confirms that this is an issue that we should be concerned with.
## Maybe we will use density functions to solve this?
width = 0.45
thinkplot.PrePlot(2)
thinkplot.Hist(first_hist, align = 'right', width=width)
thinkplot.Hist(other_hist,align='left', width=width)
thinkplot.Show(xlabel='Weeks', ylabel = 'Frequency')

In [ ]:
## Computing the mean, standard deviation and variance
mean = live.prglngth.mean()
var = live.prglngth.var()
std = live.prglngth.std()



In [ ]:
## We need to find the effect size using Cohens effect size
def CohenEffectSize(group1,group2):
    diff = group1.mean()-group2.mean()

    var1 = group1.var()
    var2 = group2.var()

    n1,n2 = len(group1),len(group2)

    pooled_var =(n1*var1+n2*var2)/(n1+n2)
    d = diff/math.sqrt(pooled_var)
    return d

In [ ]:
## testing it out
## It worked! The book said that he got .029 which is not much of a difference!
effect_size = CohenEffectSize(firsts['prglngth'],others['prglngth'])
print(effect_size)
## This is very interesting since the effect size is in standard deviations it is saying that the difference is slightly larger than the mean.
# But there is not a noticeable difference in fact I would say that first born babies are born very close to the average time.
## According to the the wiki and confirmed on other websites we make decisions on effect size based on these metrics.
## very small (0.01) small (0.2) medium (0.5) large (0.8) Very Large (1.20) Huge (2.0)
## as we can see since our effect size was 0.029 we would fall between very small and small.
# So I would say that there is a small chance that the baby would be born later than a normal
# If the baby was their first baby there may be a small chance according to those metrics that the baby would slightly late.